In [ ]:
import os
import workflow_utils_v3
import sys

from workflow_utils_v3.FileDirectory import Directory

dirs = Directory(rootpath = '/home/mgolub4/DLproj/MLTO_2024/')

# Sets directory of entire package
# rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2023/'

nbpath = os.path.join(dirs._3_Dynamic_PINN_RNN, 'PINN_training')
cp_dir = os.path.join(nbpath, 'model_CPs')


In [ ]:

# from torchsummary import summary

import pandas as pd
import numpy as np
import json
import glob
import os

# For plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from itertools import cycle
from plotly.colors import sequential, qualitative


import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.optim as optim
# import torchsummary

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.model_selection import train_test_split as TTS

In [ ]:
date = '25APR24'
fname_base = f'Dyn_PINN_v0_{date}'

In [ ]:
# dyndb_path = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready.csv'
dyndb_path = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready_coeffs_scaled.csv'
dyndb = pd.read_csv(dyndb_path)

In [ ]:
idxTr, idxRem = TTS(dyndb, stratify = dyndb['topology_family'], random_state=42, train_size = 0.8)
idxVal, idxTe = TTS(idxRem, random_state = 42, test_size=0.5)

In [ ]:
params = ['volFrac', 
        'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
        'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
        'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
        'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
        'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
        'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
        'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def minmax_scale(array):
    max = array.max()
    min = array.min()

    array = (array - min) / (max - min)

    return array, max, min

In [ ]:
class Stress_Series:
    def __init__(self, series):
        self.series = series
        self.max = series.max()
        self.min = series.min()

    def scale(self):
        return (self.series - self.min) / (self.max - self.min)

In [ ]:
class PINN_Dataset(Dataset):
    def __init__(self, params, split_dataframe,
                 feat_vec_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/voxel_embedding_feature_maps', 
                 stress_series_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/stress_series_data', 
                 stress_ser_suffix = '_proct_gaus_btrlp_fftlp',
                 scale_coeffs_by=[1e9, 1e11, 1, 1, 1],
                 predicted_parameters=True,
                  ):
        self.df = split_dataframe
        self.featvec_dir = feat_vec_directory # for pulling the feature vectors
        self.stress_ser_dir = stress_series_directory # for pulling the time series files
        self.params = params
        self.predicted_parameters = predicted_parameters
        # self.const_eqn_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]
        self.const_eqn_params = ['A_opt scaled', 'B_opt scaled', 'C_opt scaled', 'm_opt scaled', 'n_opt scaled',]

        self.stress_ser_suffix = stress_ser_suffix
        self.scale_coeffs_by = scale_coeffs_by

        # self.scaler = MinMaxScaler()


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        dyn_series_fname = self.df['dyn_file_name_original'].iloc[idx]

        sig_pl = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['plateau_stress_g scaled'].values[0].astype(np.float32)
        W = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['energy_absorbed_g scaled'].values[0].astype(np.float32)

        # feature vector from convolutional neural network convolutional layers output
        featvec_fname = self.df['conv_feat_vec'].iloc[idx] + '.npy'
        featvec_path = os.path.join(self.featvec_dir, featvec_fname)
        featvec = np.load(featvec_path)
        featvec = np.squeeze(featvec, axis=0).astype(np.float32)

        # constitutive equation parameters
        constit_eqn_coeffs = np.asarray(self.df[self.const_eqn_params].iloc[idx])

        for i, scaler in enumerate(self.scale_coeffs_by):
            constit_eqn_coeffs[i] = constit_eqn_coeffs[i] / scaler
            


        # predicted parameters
        if self.predicted_parameters:
            paramvec = np.asarray([self.df[f'pred {par}'].iloc[idx] for par in self.params]).astype(np.float32)
        else:
            paramvec = np.asarray([self.df[f'{par}'].iloc[idx] for par in self.params]).astype(np.float32)

        # stress_series -- for now (April 24), Imma use the truncated datasets, because I think padded batches for RNNs in pytorch will take care of differing lengths
        stress_ser_fname = dyn_series_fname + self.stress_ser_suffix
        stress_ser_path = os.path.join(self.stress_ser_dir, stress_ser_fname+'.csv')
        stress_series = np.asarray(pd.read_csv(stress_ser_path)['stress_bottom_gsreg']).astype(np.float32)
        
        # stress_series = Stress_Series(stress_series)
        # scaled_stress_series = stress_series.scale()
        # stress_series_dic = {'stress_series': scaled_stress_series, 'max': stress_series.max, 'min': stress_series.min}

        strain = np.asarray(pd.read_csv(stress_ser_path)['Strain']).astype(np.float32)
                   

        return featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain, stress_series #stress_series_dic
        """
        inputs:
        - featvec        - paramvec
        targets:
        - stress_series        - constit_eqn_coeffs        - W        - sig_pl        other(?):        - strain --> needed for later
        """



In [ ]:
def PINN_loss(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs_data, W_data, sig_pl_data, strain):
    global strspred_glob, strn_glob, strs_start_glob, strs_const_eqn_glob
    strn_glob = strain
    offset=-0.01
    
    A_pred = constit_eqn_coeffs_pred[:, 0]
    B_pred = constit_eqn_coeffs_pred[:, 1]
    C_pred = constit_eqn_coeffs_pred[:, 2]
    m_pred = constit_eqn_coeffs_pred[:, 3]
    n_pred = constit_eqn_coeffs_pred[:, 4]

    if len(stress_series_pred.shape) == 3:
        strspred_glob = stress_pred = torch.squeeze(stress_series_pred, axis=2)
    else:
        strspred_glob = stress_pred = stress_series_pred

    

    stress_series_constit_eqn = A_pred * (strain.T + offset)**m_pred + B_pred*((strain.T + offset)/(C_pred-(strain.T + offset)))**n_pred

    strs_const_eqn_glob = stress_series_constit_eqn = stress_series_constit_eqn.T

    sig_pl_pred = torch.mean(stress_series_constit_eqn[:,200:400], dim=1)
    sig_pl_data = sig_pl_data
    # print('sig_pl data, pred shape:')
    # print(sig_pl_data.shape)
    # print(sig_pl_pred.shape)
    

    strs_start_glob = stress_start = int(-1*offset*1e3) # ensures the calculation starts at the right point of the stress series
    W_pred = torch.trapz(stress_pred[:, stress_start:], strain[:, stress_start:], dim=1)
    W_data = W_data
    # print('W data, pred shape:')
    # print(W_data.shape)
    # print(W_pred.shape)


    loss_data_1 = nn.L1Loss()(sig_pl_pred, sig_pl_data) + nn.L1Loss()(W_pred, W_data)

    loss_data_2 = nn.L1Loss()(constit_eqn_coeffs_pred, const_eqn_coeffs_data)

    loss_physics = nn.L1Loss()(stress_pred, stress_series_constit_eqn)

    total_loss = loss_data_1 + loss_data_2 + loss_physics

    return total_loss, loss_data_1, loss_data_2, loss_physics

In [ ]:
# trdata.__getitem__(0)

In [ ]:
# return featvec, paramvec, stress_series, constit_eqn_coeffs, W, sig_pl, strain
# return nonseries_list, padded_strain_ser, padded_stress_ser

from torch.nn.utils.rnn import pad_sequence

def padded_collate(batch):
    stress_ser = [torch.tensor(item[-1], requires_grad=True) for item in batch]
    strain_ser = [torch.tensor(item[-2], requires_grad=True) for item in batch]
    nonserbatch = [item[0:-2] for item in batch] # all until last element (stress series) of list given by PINN_Dataset

    nonseries_list = []

    # nonser_len = len(nonserbatch)
    for i in range(5):
        data_list = []
        for data in nonserbatch:
            # print(type(data), len(data))
            data_list.append(data[i])
        data_list = torch.tensor(np.asarray(data_list), requires_grad=True)
        nonseries_list.append(data_list)

    
    padded_stress_ser = pad_sequence(stress_ser, batch_first=True, padding_value=0)
    padded_strain_ser = pad_sequence(strain_ser, batch_first=True, padding_value=0)


    return nonseries_list, padded_strain_ser, padded_stress_ser

In [ ]:
batch_size = 2

trdata = PINN_Dataset(params, idxTr)
trloader = DataLoader(trdata, batch_size=batch_size, collate_fn = padded_collate, shuffle=True)

valdata = PINN_Dataset(params, idxVal)
valloader = DataLoader(valdata, batch_size=batch_size, collate_fn = padded_collate, shuffle=True)

tedata = PINN_Dataset(params, idxTe)
teloader = DataLoader(tedata, batch_size=batch_size, collate_fn = padded_collate, shuffle=False)

In [ ]:
# # for i in range(5):
# #     print(next(iter(trloader))[0][i].shape)
# feat = next(iter(trloader))[0][0]
# pars = next(iter(trloader))[0][1]
# # torch.cat([feat,pars], dim=1).shape

In [ ]:
# # print([i.shape for i in next(iter(trloader))[1]])
# for part in next(iter(trloader)):
#     print(type(part))
#     if isinstance(part, list):
#         print(len(part))
#     if isinstance(part, torch.Tensor):
#         print(part.shape)

# for part in next(iter(trloader))[0]:
#     print(part.shape)

In [ ]:
# class Dynamic_Stress_PINN(nn.Module):
    
#     def __init__(self, params, hidden_size=128, num_lstm_layers=4, lstm_output_dim=1):
#         numparams = len(params)
#         input_vec_dim = 1024 + numparams
#         linear_out_dims = 5
#         # self.series_in_dim = series_input_dim
#         # self.hidden_size = hidden_size
#         # self.num_lstm_layers = num_lstm_layers
#         # self.lstm_output_dim = lstm_output_dim
#         super(Dynamic_Stress_PINN, self).__init__()

#         self.stress_ser_predictor = nn.LSTM(input_vec_dim, hidden_size, num_lstm_layers, batch_first=True)
#         self.lstm_linear = nn.Sequential(nn.Linear(hidden_size, lstm_output_dim), nn.ReLU())


#         self.constit_eqn_coeff_predictor = nn.Sequential(
#             nn.Linear(input_vec_dim, 1024),
#             nn.Linear(1024, 512),nn.ReLU(),
#             nn.Linear(512, 256),nn.ReLU(),
#             nn.Linear(256, 128),nn.ReLU(),
#             nn.Linear(128, linear_out_dims)
#         )


#     def forward(self, feature_vec, property_vec):
#         input_vec = torch.cat([feature_vec, property_vec], dim=1)

#         stress_ser, _ = self.stress_ser_predictor(input_vec) # MAKE SURE YOU KNOW IT KNOWS WHEN TO STOP... PROBABLY TAKEN CARE OF BY PADDED SET
#         # stress_ser = self.lstm_linear(stress_ser)

#         constit_eqn_coeffs = self.constit_eqn_coeff_predictor(input_vec)

#         return stress_ser, constit_eqn_coeffs


# # return stress_series, featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain


In [ ]:
class Dynamic_Stress_PINN(nn.Module):
    
    def __init__(self, params, hidden_size=256, num_lstm_layers=4, lstm_output_dim=1):
        numparams = len(params)
        input_vec_dim = 1024 + numparams
        linear_out_dims = 5
        # self.series_in_dim = series_input_dim
        self.hidden_size = hidden_size
        self.num_lstm_layers = num_lstm_layers
        self.lstm_output_dim = lstm_output_dim
        super(Dynamic_Stress_PINN, self).__init__()

        self.stress_ser_predictor = nn.LSTM(input_vec_dim, hidden_size, num_lstm_layers, batch_first=True)
        self.lstm_linear = nn.Sequential(nn.Linear(hidden_size, lstm_output_dim), nn.ReLU())


        self.constit_eqn_coeff_predictor = nn.Sequential(
            nn.Linear(input_vec_dim, 1024),
            nn.Linear(1024, 512),nn.ReLU(),
            nn.Linear(512, 256),nn.ReLU(),
            nn.Linear(256, 128),nn.ReLU(),
            nn.Linear(128, linear_out_dims)
        )


    def forward(self, feature_vec, property_vec, strain_series):
        input_vec = torch.cat([feature_vec, property_vec], dim=1)

        batch_size = strain_series.size(0)
        h = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        c = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        
        stress_ser = []

        for i in range(strain_series.size(1)):
            stress, (h, c) = self.stress_ser_predictor(input_vec, (h, c))
            stress = self.lstm_linear(stress)
            stress_ser.append(stress)
       
        stress_ser = torch.stack(stress_ser, dim=1)

        constit_eqn_coeffs = self.constit_eqn_coeff_predictor(input_vec)

        return stress_ser, constit_eqn_coeffs


# return stress_series, featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain


In [ ]:
# torch.Tensor()
# torch.tensor(np.array([[0,0,0], [0,1,0]]))

In [ ]:
pinn = Dynamic_Stress_PINN(params).to(device)

In [ ]:
# dataset returns-->         featvec, paramvec, constit_eqn_coeffs, W, sig_pl, || strain,            || stress_series,
# dataloader returns -->     ^--------------nonseries_list------------------^  || padded_strain_ser, || padded_stress_ser


def pinn_train(pinn, dataloader, loss_func=PINN_loss, optimizer=optim.Adam(pinn.parameters(), lr=0.0001)):
    pinn.train()  # Set the model to training mode
    # running_loss = 0.0
    total_run_loss = 0.0
    dyn_param_run_loss = 0.0
    coeff_run_loss = 0.0
    phys_run_loss = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    for non_series_data, strain_series, stress_series in pbar:
        global feature_vec, param_vec, strs_ser_glob, coeffs_glob, strn_ser_glob
        feature_vec = non_series_data[0].cuda()
        param_vec   = non_series_data[1].cuda()
        strs_ser_glob = stress_series = stress_series.cuda()  # Move inputs to GPU
        coeffs_glob = const_eqn_coeffs = non_series_data[2].cuda()

        W = non_series_data[3].cuda()
        sig_pl = non_series_data[4].cuda()
        strain_series = strn_ser_glob = strain_series.cuda()

    
        optimizer.zero_grad()
        global stress_series_pred
        global constit_eqn_coeffs_pred
        stress_series_pred, constit_eqn_coeffs_pred = pinn(feature_vec, param_vec, strain_series)
        # print(stress_series_pred.shape, constit_eqn_coeffs_pred.shape)

        loss_val, dyn_param_loss, const_eqn_coeff_loss, phys_loss = loss_func(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs, W, sig_pl, strain_series)
                                                                    # PINN_loss(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs_data, W_data, sig_pl_data, strain, offset=-0.01):
        
        loss_val.backward()
        optimizer.step()
        # running_loss += loss_val.item()
        total_run_loss += loss_val.item()
        dyn_param_run_loss += dyn_param_loss.item()
        coeff_run_loss += const_eqn_coeff_loss.item()
        phys_run_loss += phys_loss.item()
        # pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}')
        pbar.set_description(f'Losses:\tTotal - {total_run_loss / (pbar.n + 1):.4f}\tDyn Params - {dyn_param_run_loss / (pbar.n + 1):.4f}\tCoeff - {coeff_run_loss / (pbar.n + 1):.4f}\tPhysics - {phys_run_loss / (pbar.n + 1):.4f}\t')
        # print("Data index is: f{idx}")
    return running_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss

In [ ]:
def pinn_validate(pinn, dataloader, loss_func=PINN_loss):
    pinn.train()  # Set the model to training mode
    running_loss = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    with torch.no_grad():
        for non_series_data, strain_series, stress_series in pbar:
            feature_vec = non_series_data[0].cuda()
            param_vec   = non_series_data[1].cuda()
            stress_series = stress_series.cuda()  # Move inputs to GPU
            const_eqn_coeffs = non_series_data[2].cuda()

            W = non_series_data[3].cuda()
            sig_pl = non_series_data[4].cuda()
            strain_series = strain_series.cuda()

            stress_series_pred, constit_eqn_coeffs_pred = pinn(feature_vec, param_vec, strain_series)

            loss_val, dyn_param_loss, const_eqn_coeff_loss, phys_loss = loss_func(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs, W, sig_pl, strain_series)
            # return total_loss, loss_data_1, loss_data_2, loss_physics
            
            running_loss += loss_val.item()
            pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}\t ')
        return running_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss


In [ ]:
# Model and training hyper(?)parameters

EPOCHS = 125

lossfunc = torch.nn.L1Loss() # this is MAE loss
lossfunc_name = 'MAE'
optimizer = optim.Adam(pinn.parameters(), lr=0.001)

cp_dir = os.path.join(nbpath, 'model_CPs')

cp_name = f'CP_{fname_base}.pth'
best_weights_path = os.path.join(cp_dir, cp_name)
print(best_weights_path)

In [ ]:
patience = 75

min_val_loss = float('inf')
best_val_loss = float('inf')
early_stop_counter = 0
# earlystop_min_delta = 0.000075
earlystop_min_delta = 0.00075 # For L1Loss (MAE)

# os.makedirs(best_weights_path, exist_ok=True)
best_epoch = 0

train_losses = []
val_losses = []

epochs_completed=0

In [ ]:
# train_loss

In [ ]:

# return running_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss
lossfunc = PINN_loss
try:

    for epoch in range(EPOCHS):
        # Train the model
        train_loss = pinn_train(pinn, trloader, lossfunc, optimizer)
        train_loss_val = train_loss[0]
        train_dyn_param_loss = train_loss[1]
        train_const_eqn_coeff_loss = train_loss[2]
        train_phys_loss = train_loss[3]

        # Validate the model
        val_loss = pinn_validate(pinn, valloader, lossfunc)
        val_loss_val = val_loss[0]
        val_dyn_param_loss = val_loss[1]
        val_const_eqn_coeff_loss = val_loss[2]
        val_phys_loss = val_loss[3]

        print(f'training:\ttotal loss: {train_loss_val:.5f}, dynamic param loss: {train_dyn_param_loss:.5f}\nconstitutive equation coefficient loss: {train_const_eqn_coeff_loss:.5f}, physics_loss: {train_phys_loss:.5f}')
        print(f'validation:\ttotal_loss:{val_loss_val:.5f}, dynamic param loss: {val_dyn_param_loss:.5f}\nconstitutive equation coefficient loss: {val_const_eqn_coeff_loss:.5f}, physics_loss: {val_phys_loss:.5f}')


        # Save the model's weights if validation loss is improved
        improvement_delta = best_val_loss - val_loss_val

        if val_loss_val < best_val_loss:
            pct_improved = (best_val_loss - val_loss) / best_val_loss * 100
            print(f"Val loss improved from {best_val_loss:.5f} to {val_loss:.5f} ({pct_improved:.2f}% improvement) saving model state...")
            best_val_loss = val_loss
            torch.save(pinn.state_dict(), best_weights_path)  # Save model weights to file
        else:
            print(f'Val loss did not improve from {best_val_loss:.5f}.')
            # early_stop_counter += 1  # Increment early stopping counter

        if improvement_delta > earlystop_min_delta:
            early_stop_counter = 0
        else:
            early_stop_counter +=1


        # Collect model training history
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Check for early stopping
        if early_stop_counter >= patience:
            print(f'Validation loss did not improve for {early_stop_counter} epochs. Early stopping...')
            pinn.load_state_dict(torch.load(best_weights_path))
            print(f"Model best weights restored - training epoch {best_epoch}")
            break

        print(f'Epoch [{epoch+1}/{EPOCHS}]\tTrain Loss: {train_loss_val:.5f}\tValidation Loss: {val_loss_val:.5f}')

        epochs_completed +=1


    # Load the best weights at end of training epochs
    pinn.load_state_dict(torch.load(best_weights_path))  # Load best model weights
    print(f'Training epochs completed, best model weights restored - epoch {best_epoch}')
    min_val_loss = best_val_loss

except KeyboardInterrupt:
    hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'val_loss {lossfunc_name}': val_losses}
    pinn.load_state_dict(torch.load(best_weights_path))


In [ ]:
batch = next(iter(trloader))

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(batch[-1])

In [ ]:
stress = batch[-1][0,:].numpy().reshape(-1,1)

In [ ]:
stress.shape

In [ ]:
pinn(batch[0][0].cuda(),batch[0][1].cuda(), batch[1].cuda())

In [ ]:
pinn.constit_eqn_coeff_predictor[0].weight

In [ ]:
strs_const_eqn_glob.shape

In [ ]:
strspred_glob.shape

In [ ]:
int(strs_start_glob)

In [ ]:
strspred_glob.shape

In [ ]:
strn_glob.shape

In [ ]:
torch.trapz(strspred_glob[:, 10:], strn_glob[:, 10:], dim=1)

In [ ]:
A_glob = coeffs_glob[:, 0]

In [ ]:
(A_glob*strs_ser_glob.T).shape

In [ ]:
A_glob.shape

In [ ]:
strs_ser_glob.shape

In [ ]:
loss_func=PINN_loss
optimizer=optim.Adam(pinn.parameters())
pinn.train()  # Set the model to training mode
running_loss = 0.0
batch = next(iter(trloader)) # Use tqdm for progress bars


In [ ]:
str

In [ ]:
coeffs_glob

In [ ]:
batch[0][0].shape

In [ ]:
non_series_data = batch[0]
strain_series = batch[1]
stress_series = batch[2]


In [ ]:

# global feature_vec, param_vec, strs_ser_glob, coeffs_glob
feature_vec = non_series_data[0].cuda()
param_vec   = non_series_data[1].cuda()
strs_ser_glob = stress_series = stress_series.cuda()  # Move inputs to GPU
coeffs_glob = const_eqn_coeffs = non_series_data[2].cuda()

W = non_series_data[3].cuda()
sig_pl = non_series_data[4].cuda()
strain = strain_series.cuda()


In [ ]:


optimizer.zero_grad()

stress_series_pred, constit_eqn_coeffs_pred = pinn(feature_vec, param_vec)


In [ ]:
stress_series_pred.shape

In [ ]:

loss_val, dyn_param_loss, const_eqn_coeff_loss, phys_loss = loss_func(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs,)# W, sig_pl, strain)

In [ ]:
pinn()

In [ ]:
strs_ser_glob.shape

In [ ]:
coeffs_glob

In [ ]:
constit_eqn_coeffs_pred

In [ ]:
t1 = torch.randn(200)
t2 = torch.randn(100)

t3 = torch.cat([t1, t2], dim=0)
print(t3.shape)

In [ ]:

hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'val_loss {lossfunc_name}': val_losses}

histno=1
histpath = os.path.join(nbpath,'model_jsons',f'{cp_name[:-4]}_training_history{histno}_{epochs_completed}ep.json')

pinn.eval()

with open(histpath, 'w') as f:
    json.dump(hist_dict, f)